# Find bear market start signal
- 비트 현물 홀딩시
- 언제 하락장의 시작인지 찾는다
- 일봉 기준, 어떤 MA를 터치할 때 파는 것이 수익률 가장 좋은지

In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [155]:
interval = '1d'
btc_data = pd.read_csv(f'data/btc_{interval}_data_with_ma.csv', index_col=False)
btc_data['timestamp'] = pd.to_datetime(btc_data['timestamp'])

In [156]:
ma_candidates = [x for x in range(40, 210, 10)]

In [157]:
# scenario 1: bull and bear
# bull_bear_market_df = btc_data[(btc_data['timestamp'] >= '2020-03-11') & (btc_data['timestamp'] <= '2021-07-20')]
bull_bear_market_df = btc_data[(btc_data['timestamp'] >= '2020-10-10') & (btc_data['timestamp'] <= '2021-07-20')]

# scenario 2: bear only
bear_market_df = btc_data[(btc_data['timestamp'] >= '2021-11-10') & (btc_data['timestamp'] <= '2023-01-05')]

In [158]:
# Function to calculate final value of holding BTC
def calculate_holding_value(df):
    initial_cash = 10000  # Start with $10,000 in cash

    # Buy BTC at the first available price
    initial_price = df['close'].iloc[0]
    btc_holding = initial_cash / initial_price  # Buy BTC with all available cash

    # Sell BTC at the last available price
    final_price = df['close'].iloc[-1]
    final_value = btc_holding * final_price  # Calculate the value of the BTC holding

    # print(f"Initial BTC price: {initial_price:.2f} USDT")
    # print(f"Final BTC price: {final_price:.2f} USDT")
    # print(f"Final value of holding BTC: {final_value:,.2f} USDT")
    
    return final_value

# Scenario 1: Bull and Bear market

In [159]:
baseline_value = calculate_holding_value(bull_bear_market_df)
print(f"Just holding BTC: {baseline_value:,.2f}")

Just holding BTC: 26,378.97


In [160]:
# Updated Backtest Function with RSI logic
def backtest_and_calculate_profit(_df, ma):
    df = _df.copy(deep=True)

    df['Position'] = np.where(df['close'] > df[f'MA_{ma}'], 1, 0)  # 1 for buy, 0 for sell
    df['Signal'] = df['Position'].diff()  # Buy signal when diff > 0, sell signal when diff < 0

    # Initial values
    initial_cash = 10000  # Starting with $10,000
    btc_holding = 0
    cash = initial_cash
    purchase_price = 0
    portfolio_value = 0

    # baseline: Just holding BTC
    base_price = df['close'].iloc[0]
    final_price = df['close'].iloc[-1]
    base_portfolio_value = ((final_price - base_price) / base_price + 1) * initial_cash
    print(f"Base portfilio value: {base_portfolio_value:,.2f} USDT")

    # Buy at start !!
    purchase_price = df['close'].iloc[0]
    btc_holding = cash / purchase_price  # Buy BTC with all available cash
    cash = 0  # No cash left after buying
    current_time = df['timestamp'].iloc[0]
    # print(f"{current_time}: Bought BTC at {purchase_price:,.2f}, holding {btc_holding:.4f} BTC")

    chance = 0

    for i in range(0, len(df)):
        price = df['close'].iloc[i]

        # 가격이 특정 MA 아래로 가면 선제적 매도
        # if price < df[f'MA_{ma}'].iloc[i]:
        if df['Signal'].iloc[i] == -1:
            if btc_holding > 0:
                if chance > 0: # 한번 빠진건 봐준다
                    chance -= 1
                    continue
                sell_price = price
                cash = btc_holding * sell_price  # Sell all BTC and hold cash
                btc_holding = 0  # No BTC left after selling
                current_time = df['timestamp'].iloc[i]
                print(f"{current_time}: [MA Crossed]: {ma}MA Sold BTC at {sell_price:,.2f}, now holding {cash:,.2f} USDT\n")
                break

        # Calculate current portfolio value (cash + value of BTC)
        # portfolio_value = cash + (btc_holding * price)
        # Update the portfolio value for each iteration
        # df.loc[i, 'portfolio_value'] = portfolio_value

    # Drop 'Signal' and 'Position' columns
    # df = df.drop(columns=['Signal', 'Position'])
    return cash

In [161]:
portfolio_value_by_ma = {}

for ma in ma_candidates:
    portfolio_value = backtest_and_calculate_profit(bull_bear_market_df, ma)
    portfolio_value_by_ma[ma] = portfolio_value

Base portfilio value: 26,378.97 USDT
2021-01-27 00:00:00: [MA Crossed]: 40MA Sold BTC at 30,366.15, now holding 26,888.83 USDT

Base portfilio value: 26,378.97 USDT
2021-04-18 00:00:00: [MA Crossed]: 50MA Sold BTC at 56,150.01, now holding 49,720.11 USDT

Base portfilio value: 26,378.97 USDT
2021-04-21 00:00:00: [MA Crossed]: 60MA Sold BTC at 53,787.63, now holding 47,628.25 USDT

Base portfilio value: 26,378.97 USDT
2021-04-21 00:00:00: [MA Crossed]: 70MA Sold BTC at 53,787.63, now holding 47,628.25 USDT

Base portfilio value: 26,378.97 USDT
2021-04-22 00:00:00: [MA Crossed]: 80MA Sold BTC at 51,690.96, now holding 45,771.68 USDT

Base portfilio value: 26,378.97 USDT
2021-04-24 00:00:00: [MA Crossed]: 90MA Sold BTC at 50,047.84, now holding 44,316.71 USDT

Base portfilio value: 26,378.97 USDT
2021-04-25 00:00:00: [MA Crossed]: 100MA Sold BTC at 49,066.77, now holding 43,447.99 USDT

Base portfilio value: 26,378.97 USDT
2021-05-12 00:00:00: [MA Crossed]: 110MA Sold BTC at 49,631.32, no

In [162]:
print (f"Just holding BTC baseline: {baseline_value:,.2f}")
for ma, value in portfolio_value_by_ma.items():
    print(f"Final value for ma {ma}: {value:,.2f} USDT") 

# Find the MA with the highest portfolio value
best_ma = max(portfolio_value_by_ma, key=portfolio_value_by_ma.get)
best_value = portfolio_value_by_ma[best_ma]

print(f"The best performing strategy for selling all BTC below {best_ma}MA resulted in a final portfolio value of {best_value:,.2f} USDT")

Just holding BTC baseline: 26,378.97
Final value for ma 40: 26,888.83 USDT
Final value for ma 50: 49,720.11 USDT
Final value for ma 60: 47,628.25 USDT
Final value for ma 70: 47,628.25 USDT
Final value for ma 80: 45,771.68 USDT
Final value for ma 90: 44,316.71 USDT
Final value for ma 100: 43,447.99 USDT
Final value for ma 110: 43,947.89 USDT
Final value for ma 120: 43,947.89 USDT
Final value for ma 130: 41,408.02 USDT
Final value for ma 140: 41,408.02 USDT
Final value for ma 150: 41,114.49 USDT
Final value for ma 160: 38,552.37 USDT
Final value for ma 170: 38,552.37 USDT
Final value for ma 180: 32,488.60 USDT
Final value for ma 190: 32,488.60 USDT
Final value for ma 200: 32,488.60 USDT
The best performing strategy for selling all BTC below 50MA resulted in a final portfolio value of 49,720.11 USDT


## Scenario 2: Bear market only

In [163]:
baseline_value = calculate_holding_value(bear_market_df)
print(f"Just holding BTC: {baseline_value:,.2f}")

Just holding BTC: 2,594.21


In [164]:
portfolio_value_by_ma = {}

for ma in ma_candidates:
    portfolio_value = backtest_and_calculate_profit(bear_market_df, ma)
    portfolio_value_by_ma[ma] = portfolio_value

Base portfilio value: 2,594.21 USDT
2021-11-16 00:00:00: [MA Crossed]: 40MA Sold BTC at 60,058.87, now holding 9,256.57 USDT

Base portfilio value: 2,594.21 USDT
2021-11-18 00:00:00: [MA Crossed]: 50MA Sold BTC at 56,891.62, now holding 8,768.42 USDT

Base portfilio value: 2,594.21 USDT
2021-11-22 00:00:00: [MA Crossed]: 60MA Sold BTC at 56,247.18, now holding 8,669.09 USDT

Base portfilio value: 2,594.21 USDT
2021-11-26 00:00:00: [MA Crossed]: 70MA Sold BTC at 53,726.53, now holding 8,280.60 USDT

Base portfilio value: 2,594.21 USDT
2021-11-26 00:00:00: [MA Crossed]: 80MA Sold BTC at 53,726.53, now holding 8,280.60 USDT

Base portfilio value: 2,594.21 USDT
2021-11-26 00:00:00: [MA Crossed]: 90MA Sold BTC at 53,726.53, now holding 8,280.60 USDT

Base portfilio value: 2,594.21 USDT
2021-11-26 00:00:00: [MA Crossed]: 100MA Sold BTC at 53,726.53, now holding 8,280.60 USDT

Base portfilio value: 2,594.21 USDT
2021-12-03 00:00:00: [MA Crossed]: 110MA Sold BTC at 53,601.05, now holding 8,261

In [165]:
print (f"Just holding BTC baseline: {baseline_value:,.2f}")
for ma, value in portfolio_value_by_ma.items():
    print(f"Final value for ma {ma}: {value:,.2f} USDT") 

# Find the MA with the highest portfolio value
best_ma = max(portfolio_value_by_ma, key=portfolio_value_by_ma.get)
best_value = portfolio_value_by_ma[best_ma]

print(f"The best performing strategy for selling all BTC below {best_ma} resulted in a final portfolio value of {best_value:,.2f} USDT")

Just holding BTC baseline: 2,594.21
Final value for ma 40: 9,256.57 USDT
Final value for ma 50: 8,768.42 USDT
Final value for ma 60: 8,669.09 USDT
Final value for ma 70: 8,280.60 USDT
Final value for ma 80: 8,280.60 USDT
Final value for ma 90: 8,280.60 USDT
Final value for ma 100: 8,280.60 USDT
Final value for ma 110: 8,261.26 USDT
Final value for ma 120: 7,575.62 USDT
Final value for ma 130: 7,575.62 USDT
Final value for ma 140: 7,575.62 USDT
Final value for ma 150: 7,575.62 USDT
Final value for ma 160: 7,327.96 USDT
Final value for ma 170: 7,327.96 USDT
Final value for ma 180: 7,327.96 USDT
Final value for ma 190: 7,198.06 USDT
Final value for ma 200: 7,198.06 USDT
The best performing strategy for selling all BTC below 40 resulted in a final portfolio value of 9,256.57 USDT
